# Dimensión fecha

### Librerías necesarias

In [2]:
from datetime import date
import pandas as pd
import numpy as np
import holidays
import yaml

### Módulo de extracción

#### Creación del dataframe fecha

In [3]:
# Creación del dataframe
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='1/1/2009', freq='D')
})
dim_fecha.head()

,date
0,2005-01-01
1,2005-01-02
2,2005-01-03
3,2005-01-04
4,2005-01-05


### Módulo de transformación

In [4]:
dim_fecha["year"] = dim_fecha["date"].dt.year
dim_fecha["month"] = dim_fecha["date"].dt.month
dim_fecha["day"] = dim_fecha["date"].dt.day
dim_fecha["weekday"] = dim_fecha["date"].dt.weekday
dim_fecha["quarter"] = dim_fecha["date"].dt.quarter

dim_fecha.head()

,date,year,month,day,weekday,quarter
0,2005-01-01,2005,1,1,5,1
1,2005-01-02,2005,1,2,6,1
2,2005-01-03,2005,1,3,0,1
3,2005-01-04,2005,1,4,1,1
4,2005-01-05,2005,1,5,2,1


In [5]:
dim_fecha["day_of_year"] = dim_fecha["date"].dt.day_of_year
dim_fecha["day_of_month"] = dim_fecha["date"].dt.days_in_month
dim_fecha["month_str"] = dim_fecha["date"].dt.month_name()
dim_fecha["day_str"] = dim_fecha["date"].dt.day_name()
dim_fecha["date_str"] = dim_fecha["date"].dt.strftime("%d/%m/%Y")
dim_fecha.head()

,date,year,month,day,weekday,quarter,day_of_year,day_of_month,month_str,day_str,date_str
0,2005-01-01,2005,1,1,5,1,1,31,January,Saturday,01/01/2005
1,2005-01-02,2005,1,2,6,1,2,31,January,Sunday,02/01/2005
2,2005-01-03,2005,1,3,0,1,3,31,January,Monday,03/01/2005
3,2005-01-04,2005,1,4,1,1,4,31,January,Tuesday,04/01/2005
4,2005-01-05,2005,1,5,2,1,5,31,January,Wednesday,05/01/2005


In [6]:
co_holidays = holidays.CO(language="es")
dim_fecha["is_Holiday"] = dim_fecha["date"].apply(lambda x:  x in co_holidays)
dim_fecha["holiday"] = dim_fecha["date"].apply(lambda x: co_holidays.get(x))
dim_fecha["saved"] = date.today()
dim_fecha["weekend"] = dim_fecha["weekday"].apply(lambda x: x>4)
dim_fecha.head()

,date,year,month,day,weekday,quarter,day_of_year,day_of_month,month_str,day_str,date_str,is_Holiday,holiday,saved,weekend
0,2005-01-01,2005,1,1,5,1,1,31,January,Saturday,01/01/2005,True,Año Nuevo,2024-10-21,True
1,2005-01-02,2005,1,2,6,1,2,31,January,Sunday,02/01/2005,False,None,2024-10-21,True
2,2005-01-03,2005,1,3,0,1,3,31,January,Monday,03/01/2005,False,None,2024-10-21,False
3,2005-01-04,2005,1,4,1,1,4,31,January,Tuesday,04/01/2005,False,None,2024-10-21,False
4,2005-01-05,2005,1,5,2,1,5,31,January,Wednesday,05/01/2005,False,None,2024-10-21,False


### Módulo de carga a la bodega de datos